In [1]:
# import python_example as m

In [2]:
m.add(1, 2) == 3

NameError: name 'm' is not defined

In [ ]:
assert m.subtract(1, 2) == -1

In [1]:
%cd ..

/home/jovyan/work


In [2]:
%load_ext autoreload
%autoreload 3

import numpy as np
import pandas as pd
from gaccord import GraphicalAccord
from utils import standardize, partial_corr, pseudo_BIC
from generate_graphs import generate_erdos_renyi, generate_data

np.random.seed(3)

# true precision matrix
Theta = np.zeros((3,3), order="C")
Theta[0, 1] = 2.1
Theta[1, 2] = -2.1
Theta += Theta.T
np.fill_diagonal(Theta, 3)
Partial_corr = partial_corr(Theta)

# true covariance matrix
Sigma = np.linalg.inv(Theta)

# observations
X = np.random.multivariate_normal([0]*3, Sigma, 500)

# sample covariance matrix
S = np.cov(X, rowvar=False)

In [3]:
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.01 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 5)
lams

array([ 0.11686885,  0.36957176,  1.16868854,  3.69571765, 11.68688536])

In [4]:
model = GraphicalAccord(lam1=0.5, lam2=0, epstol=1e-7, maxitr=500, penalize_diag=True)
model.fit(X)
Theta_hat = model.precision_.toarray()
Partial_corr_hat = partial_corr(Theta_hat)

lam_mat = np.where(Theta_hat != 0, 0, 100.0)
model = GraphicalAccord(lam1=lam_mat, lam2=0, epstol=1e-7, maxitr=500, penalize_diag=True)
model.fit(X)
Theta_hat_debiased = model.precision_.toarray()
Partial_corr_hat_debiased = partial_corr(Theta_hat_debiased)
print('True partial correlation matrix:\n %s\n' % (Partial_corr))
print('Estimated partial correlation matrix:\n %s\n' % (Partial_corr_hat))
print('Debiased partial correlation matrix:\n %s\n' % (Partial_corr_hat_debiased))

print('Frobenius norm diff (biased):\n %s\n' % (np.linalg.norm(Partial_corr - Partial_corr_hat)))
print('Frobenius norm diff (debiased):\n %s\n' % (np.linalg.norm(Partial_corr - Partial_corr_hat_debiased)))

: 

In [5]:
n, p = 500, 1000
n_prop_to_p = [0.5]
Theta, Sigma = generate_erdos_renyi(p, type='peng', edge_prob=0.01, lower_weight=0.5, upper_weight=1.0, spread_diag=[1, np.sqrt(3)], random_state=2023)
Rho = partial_corr(Theta)

Xs = generate_data(p, n_prop_to_p, Sigma, N=1, random_state=2023)
X = Xs[0]

In [6]:
model = GraphicalAccord(lam1=0.12, lam2=0, epstol=1e-7, maxitr=500, penalize_diag=True)
model.fit(X)
Theta_hat = model.precision_.toarray()
Partial_corr_hat = partial_corr(Theta_hat)

: 